- Récupération des informations sur le candidat au prêt

In [91]:
nouvel_input = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 10000,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

- Utilisation de bigml pour récupérer le modèles d'apprentissage

In [92]:
from pandas import DataFrame, read_csv, Series
from bigml.api import BigML
api = BigML(project="project/5d94a42042129f2b0500005c")
model = api.get_ensemble("ensemble/5db162ae7811dd0557000e0e")

- Mise en format pour faciliter la manipulation

In [93]:
df = DataFrame.from_dict([nouvel_input])
df

,RevolvingUtilizationOfUnsecuredLines,NumberOfDependents,DebtRatio,age,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate
0,0.017036,1,10000,42,6,1,1,0,0


- Modification des contenus vide (NA)

In [94]:
if 'MonthlyIncome' in df.columns:
    df['MonthlyIncome_sansna'] = df['MonthlyIncome'].fillna(0)

- Calcul de nouvelle features

In [95]:
df['Sum'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTime60-89DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate']

In [96]:
if 'MonthlyIncome' in df.columns:
    df['IncomePerPerson'] = df['MonthlyIncome']/(df['NumberOfDependents']+1)
    df

In [97]:
df

,RevolvingUtilizationOfUnsecuredLines,NumberOfDependents,DebtRatio,age,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,Sum
0,0.017036,1,10000,42,6,1,1,0,0,1


- Exportation des dataframe vers des fichiers csv pour envoi à bigml

In [98]:
input_to_send = df.to_dict('records')[0]
input_to_send

{'RevolvingUtilizationOfUnsecuredLines': 0.01703559,
 'NumberOfDependents': 1,
 'DebtRatio': 10000,
 'age': 42,
 'NumberOfOpenCreditLinesAndLoans': 6,
 'NumberRealEstateLoansOrLines': 1,
 'NumberOfTime30-59DaysPastDueNotWorse': 1,
 'NumberOfTime60-89DaysPastDueNotWorse': 0,
 'NumberOfTimes90DaysLate': 0,
 'Sum': 1}

- Envoi de l'input et récupération du résultat

In [99]:
prediction = api.create_prediction(model, input_to_send)
api.ok(prediction)

True

- Récupération du résultat

In [100]:
prediction['object']['probabilities']

[['0', 0.96086], ['1', 0.03914]]

In [101]:
probability = int(prediction['object']['probabilities'][0][1]*100)

- Résultat du modèle

In [102]:
print(f"Le modèle d'apprentissage estime que ce candidat à {probability}% de chance de bien rembourser le prêt.")

Le modèle d'apprentissage estime que ce candidat à 96% de chance de bien rembourser le prêt.
